In [3]:
import pandas as pd
import matplotlib.pyplot as plt

import cvxpy
import time
import numpy as np
import multiprocessing as mp
from tqdm import tqdm

In [4]:
import io_utils
import kernels
import algos
from importlib import reload
reload(kernels)
reload(io_utils)
reload(algos)

<module 'algos' from '/home/victor/Data/Mines-ParisTech/M2/mva/S2/km/kernel-methods/algos.py'>

### Create experiment and load datasets

In [5]:
np.random.seed(42)

exp = io_utils.Experiment()
exp.create_new_experiment()

### Load previous experiment

In [6]:
kernels_wd_train = exp.load('kernels/SS_5_03.0')

100%|██████████| 3/3 [00:28<00:00,  9.64s/it]


In [8]:
# Takes ~ 1min
for kn in kernels_wd_train:
    print("Minimum vp is {:.3f}".format(np.min(np.linalg.eigvals(kn))))

Minimum vp is -0.000+0.000j
Minimum vp is -0.000+0.000j
Minimum vp is -0.000+0.000j


### Cross validation

In [9]:
reload(algos)
TRIALS = 10

lambda_vals = np.logspace(-2, 0, TRIALS)

krr = algos.KernelRidgeRegression()
# svm = algos.SVM()

krr.cross_validate(exp, kernels_wd_train, lambda_vals)

  0%|          | 0/5 [00:00<?, ?it/s]

Cross validation with 5 slices. Training set: 1600, Validation set: 400


100%|██████████| 5/5 [01:31<00:00, 18.21s/it]

Best parameter: 0.5994842503189409
Score 1: 0.631
Score 2: 0.6405000000000001
Score 3: 0.7265


8

### Compute substring kernel for all datasets

In [5]:
# kern_ss = kernels.WeightedDegreeKernel(9)
kern_ss = kernels.SubstringKernel(3, 0.1)

In [ ]:
kernels_wd_train = []

with mp.Pool(9) as pool:
    # Compute kernels


    for n in range(2,3):
        print("Processing dataset {}".format(n))
        n_train = 400#exp.raw[n].shape[0]

        t = time.time()
        args = [(exp.raw[n][i], exp.raw[n][j]) for i in range(n_train) for j in range(i, n_train) ]
        result = pool.map(kern_ss.computeKernel, args)
        e = time.time()
        print("Kernel computation took {:.2f} seconds".format(e-t))

        count = 0
        K = np.zeros((n_train, n_train))
        for i in range(n_train):
            for j in range(i, n_train):
                K[i,j] = result[count]
                count +=1

        for i in range(n_train):
            for j in range(0, i):
                K[i,j] = K[j,i]

        kernels_wd_train.append(K.copy())


In [ ]:
np.min(np.linalg.eigvals(kernels_wd_train[0]))

### If needed, save experiment

In [ ]:
kernel_name = "wd_7_02"
exp.save('kernels/{}'.format(kernel_name), kernels_train, kernels_val, kernels_test)

In [ ]:
for i in range(3):
    np.savetxt("kernels/substring_7_02_trainval_{}.txt".format(i), kernels_wd_train[i])

### (Optional) Compute matrices needed for testing 

In [6]:
# Test
exp.load_all_test_datasets()

kernels_test = []
with mp.Pool(9) as pool:
    for i in range(3):    
        print("Processing dataset {}".format(i))
        n1, n2 = exp.raw[i].shape[0], exp.raw_test[i].shape[0]
        args = [(exp.raw[i][j], exp.raw_test[i][k]) for j in range(n1) for k in range(n2) ]
        result = pool.map(kern_ss.computeKernel, args)

        kernel_test = np.zeros((n1, n2))

        count = 0
        for k in range(n1):
            for l in range(n2):
                kernel_test[k,l] = result[count]
                count +=1

        kernels_test.append(kernel_test.copy())

Processing dataset 0
Processing dataset 1
Processing dataset 2


In [ ]:
# for i in range(3):
#     np.savetxt('kernels/substring_3_01_train_test_{}.txt'.format(i), kernels_test[i])

## Start here to load saved kernels

### Linear kernel

In [7]:
linear_kernel = kernels.LinearKernel()
K_linear_train = []
for i in range(3):
    K_linear_train.append(linear_kernel.computeVectorizedKernel(exp.feats[i], exp.feats[i]))


### Gaussian kernel

In [8]:
gaussian_kernel = kernels.GaussianKernel(1)
K_gauss_train, K_gauss_val = [], []
for i in range(3):
    K_gauss_train.append(gaussian_kernel.computeVectorizedKernel(exp.feats[i], exp.feats[i]))


In [ ]:
## Prediction 
reload(algos)
lambd_substring = 1e-1
lambd_gauss = 4e-4
lambd_linear = 1e-2
lambd_svm = 2e-2
TRIALS = 30

lambda_vals = np.logspace(-7, -3, TRIALS)

all_preds = []

krr = algos.KernelRidgeRegression()
svm = algos.SVM()

# weights = np.ones((3,2))

# for i in range(3):
#     krr.fit(K_linear_train[i], exp.labels_train[i], lambd_substring)
#     error_1 = krr.evaluatePerformance(kernels_val[i], exp.labels_val[i])
#     predictions_1 = np.einsum('i, ij->j', krr.alpha, kernels_val[i])
#     w_1 = 1-error_1
    
krr.fit(kernels_wd_train[i], exp.labels_train[i], lambd_linear)
error_4 = krr.evaluatePerformance(kernels_wd_val[i], exp.labels_val[i])
w_4 = (1-error_4)
predictions_4 = np.einsum('i, ij->j', krr.alpha, kernels_wd_val[i])

#     krr.fit(K_gauss_train[i], exp.labels_train[i], lambd_gauss)
#     error_3 = krr.evaluatePerformance(K_gauss_val[i], exp.labels_val[i])
#     w_3 = (1-error_3)
#     predictions_3 = np.einsum('i, ij->j', krr.alpha, K_gauss_val[i])
    
#     n = K_gauss_train[i].shape[0]
#     training_labels_ret = exp.labels_train[i].copy()
#     training_labels_ret[training_labels_ret == -1 ] = 0
        
#     global_error =  w_1 / (w_1 +w_3+w_4) *predictions_1 + w_3 / (w_1 +w_3+w_4) * predictions_3 + w_4 / (w_1 +w_3+w_4) * predictions_4
    
#     print(np.round(1-np.sum(np.sign(global_error) != np.sign(exp.labels_val[i])) / exp.labels_val[i].shape[0],3))
#     print("----")

# Test
#     pred_test = krr.predict(kernel_tests[i])
#     all_preds.append(pred_test.copy())

    

In [28]:
kernels_normalized = []
for n in range(3):
    tmp = np.zeros_like(kernels_wd_train[0])
    for i in range(kernels_wd_train[0].shape[0]):
        for j in range(kernels_wd_train[0].shape[0]):
            tmp[i,j] = kernels_wd_train[n][i,j] / (np.sqrt(kernels_wd_train[n][i,i] * kernels_wd_train[n][j,j]))
    kernels_normalized.append(tmp.copy())